In [34]:
# score function to evaluate motif 
def Score(Motif):
    a = len(Motif)
    b = len(Motif[0])
    count = 0
    for i in range(b):
        countA = 0
        countC = 0
        countG = 0
        countT = 0
        for j in range(a):
            if Motif[j][i] == 'A':
                countA += 1
            elif Motif[j][i] == 'C':
                countC += 1
            elif Motif[j][i] == 'G':
                countG += 1
            elif Motif[j][i] == 'T':
                countT += 1
        count = count+ countA + countC + countG + countT - max(countA, countC, countG, countT)
    return count

# profile functions: get the probability profile for motifs
def profile(motif):
    a = len(motif)
    b = len(motif[0])
    prob = {'A': [0]*b, 'C': [0]*b, 'G': [0]*b, 'T': [0]*b} 
    for i in range(b):
        countA = 0
        countC = 0
        countG = 0
        countT = 0
        for m in range(a):
            if motif[m][i] == 'A':
                countA += 1
            elif motif[m][i] == 'C':
                countC += 1
            elif motif[m][i] == 'G':
                countG += 1
            elif motif[m][i] == 'T':
                countT += 1
        sum_count = countA + countC+ countG+ countT
        prob['A'][i] = countA/sum_count
        prob['C'][i] = countC/sum_count
        prob['G'][i] = countG/sum_count
        prob['T'][i] = countT/sum_count
    return prob

# modified profile functions with Laplace rule: get the probability profile for motifs, add pseodocount
def modified_profile(motif, pseudocount):
    a = len(motif)
    b = len(motif[0])
    prob = {'A': [0]*b, 'C': [0]*b, 'G': [0]*b, 'T': [0]*b} 
    for i in range(b):
        countA = pseudocount
        countC = pseudocount
        countG = pseudocount
        countT = pseudocount
        for j in range(a):
            if motif[j][i] == 'A':
                countA += 1
            elif motif[j][i] == 'C':
                countC += 1
            elif motif[j][i] == 'G':
                countG += 1
            elif motif[j][i] == 'T':
                countT += 1
        sum_count = countA + countC+ countG+ countT
        prob['A'][i] = countA/sum_count
        prob['C'][i] = countC/sum_count
        prob['G'][i] = countG/sum_count
        prob['T'][i] = countT/sum_count
    return prob

# funcitions return the most probable kmer according to the profile
def ProfileMostProbableKmer(text, k, profile):
    final = 0
    a = len(text)
    final_text = text[:k]
    for i in range(a-k+1):
        prob = 1
        pattern = text[i:i+k]
        for j in range(k):
            if pattern[j] == 'A':
                prob = prob * profile['A'][j]
            elif pattern[j] == 'C':
                prob = prob * profile['C'][j]
            elif pattern[j] == 'G':
                prob = prob * profile['G'][j]
            elif pattern[j] == 'T':
                prob = prob * profile['T'][j]
        if prob > final:
            final = prob
            final_text= text[i: i+k]
    return final_text 



In [104]:
import random
random.randint(0, 5)


1

In [100]:
random.randint(0, 5)

2

In [30]:
# randomlty generate motif from each DNA sequence
def gen_motifs(Dna, k):
    a = len(Dna[1])
    motifs = []
    for i in Dna:
        b = random.randint(0, a-k)
        motif = i[b: b+k]
        motifs.append(motif)
    return motifs

def motifs_from_profile(dna, k, profile):
    motifs = []
    for i in dna:
        motif = ProfileMostProbableKmer(i, k, profile)
        motifs.append(motif)
    return motifs



In [90]:
def RandomizedMotifSearch(Dna, k, t):
    motifs = gen_motifs(Dna, k)
    best_motifs = gen_motifs(Dna, k)
    while True:
        new_profile = modified_profile(motifs,1)
        motifs = motifs_from_profile(Dna, k, new_profile)
        if Score(motifs) < Score(best_motifs):
            best_motifs = motifs
        else:
            return best_motifs
        
        
    

In [286]:
dna = ['ATGAGGTC',
'GCCCTAGA',
'AAATAGAT',
'TTGTGCTA']

M = ['GTC',
'CCC',
'ATA',
'GCT']

In [287]:
N = 1000
for i in range(N+1):
    M = RandomizedMotifSearch(dna, 3, 4)
    if Score(M) < Score(bMotifs):
         bMotifs = M
    else:
        bestMotifs = bMotifs
        
bestMotifs

['GAG', 'TAG', 'TAG', 'TTG']

In [84]:
M = RandomizedMotifSearch(dna, 8, 5)
bMotifs = M
N = 1000
for i in range(N+1):
    M = RandomizedMotifSearch(dna, 8, 5)
    if Score(M) < Score(bMotifs):
         bMotifs = M
    else:
        bestMotifs = bMotifs
        
bestMotifs

['TCTCGGGG', 'CCAAGGTG', 'TACAGGCG', 'TTCAGGTG', 'TCCACGTG']

In [97]:
import sys        
lines = sys.stdin.read().splitlines()       



In [ ]:
k, t = map(int, lines[0].split(" "))
Dna = []
for i in range(1, len(lines)):
    Dna.append(lines[i])
    
    

In [139]:
def GibbsSampler(Dna, k, t, N):
    motifs = gen_motifs(Dna, k)
    best_motifs = motifs
    for j in range(N):
        i = random.randint(0, t-1)
        item = motifs[i]
        motifs.remove(item)
        new_profile = modified_profile(motifs,1)
        motif_prob = profile_prob(Dna[i], k, new_profile)
        dice = random.random()
        for m in range(len(motif_prob)-1):
            new_motif = ''
            if motif_prob[m] < dice and motif_prob[m+1] > dice:
                new_motif = Dna[i][m: m+k]
        motifs.insert(i, new_motif)
        if Score(motifs) < Score(best_motifs):
            best_motifs = motifs
    return best_motifs

In [166]:
def profile_prob(text, k, profile):
    prob_list = []
    a = len(text)
    for i in range(a-k+1):
        prob = 1
        pattern = text[i:i+k]
        for j in range(k):
            if pattern[j] == 'A':
                prob = prob * profile['A'][j]
            elif pattern[j] == 'C':
                prob = prob * profile['C'][j]
            elif pattern[j] == 'G':
                prob = prob * profile['G'][j]
            elif pattern[j] == 'T':
                prob = prob * profile['T'][j]
        prob_list.append(prob)
    prob_array = np.array(prob_list)/sum(prob_list)
    prob_dice = [0]
    sum_prob = 0
    for i in prob_array:
        sum_prob += i
        prob_dice.append(sum_prob)
    return prob_dice

In [110]:
import numpy as np
ab = np.array([1, 2, 3, 4])
sum(ab)

10

In [119]:
dna = ['CGCCCCTCTCGGGGGTGTTCAGTAACCGGCCA',
'GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG',
'TAGTACCGAGACCGAAAGAAGTATACAGGCGT',
'TAGATCAAGTTTCAGGTGCACGTCGGTGAACC',
'AATCCACCAGCTCCACGTGCAATGTTGGCCTA']

In [155]:
motifs = gen_motifs(dna, 8)
motifs

['TTCAGTAA', 'CGAGGTAT', 'ACCGAAAG', 'CAGGTGCA', 'TGTTGGCC']

In [156]:
i = random.randint(0, 5)
item = motifs[i]
item

'CAGGTGCA'

In [157]:
temp_motifs = motifs.remove(item)
temp_motifs

In [159]:
motifs

['TTCAGTAA', 'CGAGGTAT', 'ACCGAAAG', 'TGTTGGCC']

In [160]:
new_profile = modified_profile(motifs,1)
new_profile

{'A': [0.25, 0.125, 0.25, 0.25, 0.25, 0.25, 0.5, 0.25],
 'C': [0.25, 0.25, 0.375, 0.125, 0.125, 0.125, 0.25, 0.25],
 'G': [0.125, 0.375, 0.125, 0.375, 0.5, 0.25, 0.125, 0.25],
 'T': [0.375, 0.25, 0.25, 0.25, 0.125, 0.375, 0.125, 0.25]}

In [214]:
def GibbsSampler(Dna, k, t, N):
    motifs = gen_motifs(Dna, k)
    best_motifs = motifs
    for j in range(N):
        i = random.randint(0, t-1)
        item = motifs[i]
        motifs.remove(item)
        new_profile = modified_profile(motifs,1)
        motif_prob = profile_prob(Dna[i], k, new_profile)
        dice = random.random()
        for m in range(len(motif_prob)-1):
            new_motif = ''
            if motif_prob[m+1] > dice :
                new_motif = Dna[i][m: m+k]
                break
        motifs.insert(i, new_motif)
        if Score(motifs) < Score(best_motifs):
            best_motifs = motifs
            print(best_motifs)
    return best_motifs

In [241]:
GibbsSampler(dna, 8, 5, 1000)

['TGTTCAGT', 'AGTGCCAA', 'AGTACCGA', 'TAGATCAA', 'TCCACCAG']

In [249]:
bMotifs = gen_motifs(dna, 5)
N = 500
for i in range(N+1):
    M = GibbsSampler(dna, 8, 5, 100)
    if Score(M) < Score(bMotifs):
         bMotifs = M
    else:
        bestMotifs = bMotifs
        
bestMotifs

['TCTCGGGG', 'CCAAGGTG', 'TACAGGCG', 'TTCAGGTG', 'TCCACGTG']

In [273]:
nMotifs = gen_motifs(dna, 8)
for i in range(30):
    M = GibbsSampler(dna, 8, 5, 100)
    if Score(M) < Score(nMotifs):
         nMotifs = M
    else:
        bMotifs = nMotifs
        
        
N = 100
for i in range(N+1):
    M = GibbsSampler(dna, 8, 5, 100)
    if Score(M) < Score(bMotifs):
         bMotifs = M
    else:
        bestMotifs = bMotifs
        
bMotifs

['TCTCGGGG', 'TGTAAGTG', 'TACAGGCG', 'TTCAGGTG', 'TCCACGTG']

In [267]:
dna = ['CGCCCCTCTCGGGGGTGTTCAGTAACCGGCCA',
'GGGCGAGGTATGTGTAAGTGCCAAGGTGCCAG',
'TAGTACCGAGACCGAAAGAAGTATACAGGCGT',
'TAGATCAAGTTTCAGGTGCACGTCGGTGAACC',
'AATCCACCAGCTCCACGTGCAATGTTGGCCTA']

In [195]:
GibbsSampler(dna, 8, 5, 1000)

['TGTTCAGT', 'TGTAAGTG', 'TACAGGCG', 'TGCACGTC', 'TGCAATGT']

In [178]:
dice = random.random()
print(dice)
for m in range(len(motif_prob)-1):
    print(m, motif_prob[m])
    new_motif = ''
    if motif_prob[m+1] > dice :
        new_motif = dna[1][m: m+8]
        break
     

0.11547056670832667
0 0
1 0.007838014369693011
2 0.07054212932723711
3 0.07315480078380145
4 0.1045068582625735


'GAGGTATG'

In [284]:
with open('dataset_163_4.txt') as f:
    lines = f.readlines()
    
k, t, N = map(int, lines[0].split(" "))
dna = []
for i in range(1, len(lines)):
    dna.append(lines[i])

In [285]:
nMotifs = gen_motifs(dna, k)
for i in range(30):
    M = GibbsSampler(dna, k, t, N)
    if Score(M) < Score(nMotifs):
         nMotifs = M
    else:
        bMotifs = nMotifs
        
N = 100
for i in range(N+1):
    M = GibbsSampler(dna, k, t, N)
    if Score(M) < Score(bMotifs):
         bMotifs = M
    else:
        bestMotifs = bMotifs
        
print ('\n'.join(bestMotifs))

GCTCACCTAATCATG
ATTTTACGAATCAAC
ATTCACGTGATCAAC
GGTCACCGAATCAAG
ATTCACCGAAGTTAC
AACTACCGAATCAAC
ATTCACTCTATCAAC
ATTCAAGCAATCAAC
ATTCACCGAATCTTA
ATTCACCCCCTCAAC
ATTCACCGGCCCAAC
ATTAGTCGAATCAAC
ATTCTATGAATCAAC
GTTCACCGAATCATG
ATTCACCGAGAAAAC
TGCCACCGAATCAAC
ATTCATGAAATCAAC
ATTCACCGAATATGC
ATATCCCGAATCAAC
ATTCTGTGAATCAAC


In [257]:
dna

['TAACTCCAGAATTGATCTTCATACTAAAGCTGGGAGCAATGTTGAGAATAACCAAGAGCAAAAGTCTAGTGGTACGACACTAGCACTATCGAACCTGACCTGCGTCGTAGCCACATTTGTAAGTCTCAGTATCGAGCTGGGTGCTAACATGGTCTCGCACTATGTTGGTCACAGAAACCTCGGCAGCTGCGACGTGAGCCTAGTAGTAACATTCCCGGGTGGTATATGCGGTGATCCTAGTGGCCGCAGACAAGGGATATCACCTGAGGGGATCAGGGCGCCGTGCCCACGTGCTATTTGGCACGGCTAACTCCAGAATTGA\n',
 'TCTTCATACTAAAGCTGGGAGCAATGTTGAGAATAACCAAGAGCAAAAGTCTAGTGTTACCCTGAAAGACGGTACGACACTAGCACTATCGAACCTGACCTGCGTCGTAGCCACATTTGTAAGTCTCAGTATCGAGCTGGGTGCTAACATGGTCTCGCACTATGTTGGTCACAGAAACCTCGGCAGCTGCGACGTGAGCCTAGTAGTAACATTCCCGGGTGGTATATGCGGTGATCCTAGTGGCCGCAGACAAGGGATATCACCTGAGGGGATCAGGGCGCCGTGCCCACGTGCTATTTGGCACGGCTAACTCCAGAATTGA\n',
 'GCAGCAAACCCGCAATTGTCTCGTGCCTTTTACGATCACCAGACAGATAGAGAAGATTACCGGACTGACAAGTGAAATCCTGGCCGTAGTTAACCAGAAAGACGGCGACTGTATAAGCGCGCGCAAGTGCAATGGCGTACTGACCATACAGCCCTAAGTAACGTCGCTAAACCCTAACAGGGACCTGTATAGTTCAGTTTGGGACAGTCATAGTGAAGGGTCAGTCCCAGGGAAAAGAAAACATGCATAAAGCCGGGCCATTGTATTCGTTTATTAGGAAGAACACTACGCACCAGGAATGGGGACTCGGAAAATGTAGGAA\n',
 'GCTCCTTTAGG